In [1]:
import os
import sys

import pandas as pd 
import numpy as np

from matplotlib import pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier


from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import MinMaxScaler, RobustScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve

# import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import recall_score, f1_score, precision_score, precision_recall_curve

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/final_data_2.csv')
df.head()

,Unnamed: 0,ID,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,STABLE_WORK,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,STATUS,FEMALE,MALE,Commercial associate,Pensioner,State servant,Student,Working,Civil marriage,Married,Separated,Single / not married,Widow,Co-op apartment,House / apartment,Municipal apartment,Office apartment,Rented apartment,With parents,AGE
0,0,5008804,1,1,12.965710,1,0,1,0,0,2.0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,32
1,1,5008806,1,1,11.630709,0,0,0,0,0,2.0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,58
2,2,5008808,0,1,12.506177,0,0,0,1,1,1.0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,52
3,3,5008812,0,1,12.554967,1,1,0,0,0,1.0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,61
4,4,5008815,1,1,12.506177,1,0,1,1,1,2.0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,46


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9579 entries, 0 to 9578
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            9579 non-null   int64  
 1   ID                    9579 non-null   int64  
 2   FLAG_OWN_CAR          9579 non-null   int64  
 3   FLAG_OWN_REALTY       9579 non-null   int64  
 4   AMT_INCOME_TOTAL      9579 non-null   float64
 5   NAME_EDUCATION_TYPE   9579 non-null   int64  
 6   STABLE_WORK           9579 non-null   int64  
 7   FLAG_WORK_PHONE       9579 non-null   int64  
 8   FLAG_PHONE            9579 non-null   int64  
 9   FLAG_EMAIL            9579 non-null   int64  
 10  CNT_FAM_MEMBERS       9579 non-null   float64
 11  STATUS                9579 non-null   int64  
 12  FEMALE                9579 non-null   int64  
 13  MALE                  9579 non-null   int64  
 14  Commercial associate  9579 non-null   int64  
 15  Pensioner            

In [5]:
df.columns

Index(['Unnamed: 0', 'ID', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'AMT_INCOME_TOTAL', 'NAME_EDUCATION_TYPE', 'STABLE_WORK',
       'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS',
       'STATUS', 'FEMALE', 'MALE', 'Commercial associate', 'Pensioner',
       'State servant', 'Student', 'Working', 'Civil marriage', 'Married',
       'Separated', 'Single / not married', 'Widow', 'Co-op apartment',
       'House / apartment', 'Municipal apartment', 'Office apartment',
       'Rented apartment', 'With parents', 'AGE'],
      dtype='object')

In [6]:
target = 'STATUS'
features = ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
          'AMT_INCOME_TOTAL', 'NAME_EDUCATION_TYPE', 'STABLE_WORK',
          'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL', 'CNT_FAM_MEMBERS',
          'FEMALE', 'MALE', 'Commercial associate', 'Pensioner',
          'State servant', 'Student', 'Working', 'Civil marriage', 'Married',
          'Separated', 'Single / not married', 'Widow', 'Co-op apartment',
          'House / apartment', 'Municipal apartment', 'Office apartment',
          'Rented apartment', 'With parents', 'AGE']

In [7]:
x = df[features].to_numpy()
y = df[target].to_numpy()

In [8]:
scaler = StandardScaler()
x_norm = scaler.fit_transform(x)
x_norm.shape

(9579, 28)

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_norm, y, stratify=y, test_size=0.2, random_state=10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((7663, 28), (1916, 28), (7663,), (1916,))

Логистическая регрессия + GRID Search

In [10]:
%%time
grid = {
    "penalty": ['l1', 'l2'],
    "C": [0.001, 0.01, 0.1, 1],
    "class_weight": ["balanced", None]
#     "solver": ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
}

logreg = LogisticRegression()

logreg_gs = GridSearchCV(estimator=logreg, 
                  param_grid=grid,
                  cv=5, 
                  scoring="f1",
                  verbose=4)

logreg_gs.fit(x_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.001, class_weight=balanced, penalty=l1 ......................
[CV]  C=0.001, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, class_weight=balanced, penalty=l1 ......................
[CV]  C=0.001, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, class_weight=balanced, penalty=l1 ......................
[CV]  C=0.001, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, class_weight=balanced, penalty=l1 ......................
[CV]  C=0.001, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, class_weight=balanced, penalty=l1 ......................
[CV]  C=0.001, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.001, class_weight=balanced, penalty=l2 ......................
[CV]  C=0.001, class_weight=balanced, penalty=l2, score=0.103, total=   0.0s
[CV] C=0.001, class_weight=balanced, penalty=l2 .............

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV]  C=0.001, class_weight=None, penalty=l2, score=0.000, total=   0.0s
[CV] C=0.001, class_weight=None, penalty=l2 ..........................
[CV]  C=0.001, class_weight=None, penalty=l2, score=0.000, total=   0.0s
[CV] C=0.001, class_weight=None, penalty=l2 ..........................
[CV]  C=0.001, class_weight=None, penalty=l2, score=0.000, total=   0.0s
[CV] C=0.001, class_weight=None, penalty=l2 ..........................
[CV]  C=0.001, class_weight=None, penalty=l2, score=0.000, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l1 .............

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV]  C=0.01, class_weight=balanced, penalty=l2, score=0.100, total=   0.0s
[CV] C=0.01, class_weight=balanced, penalty=l2 .......................
[CV]  C=0.01, class_weight=balanced, penalty=l2, score=0.117, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV] . C=0.01, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV] . C=0.01, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV] . C=0.01, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV] . C=0.01, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l1 ...........................
[CV] . C=0.01, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=0.01, class_weight=None, penalty=l2 ........................

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV]  C=1, class_weight=balanced, penalty=l2, score=0.097, total=   0.1s
[CV] C=1, class_weight=balanced, penalty=l2 ..........................
[CV]  C=1, class_weight=balanced, penalty=l2, score=0.108, total=   0.0s
[CV] C=1, class_weight=None, penalty=l1 ..............................
[CV] .... C=1, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=1, class_weight=None, penalty=l1 ..............................
[CV] .... C=1, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=1, class_weight=None, penalty=l1 ..............................
[CV] .... C=1, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=1, class_weight=None, penalty=l1 ..............................
[CV] .... C=1, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=1, class_weight=None, penalty=l1 ..............................
[CV] .... C=1, class_weight=None, penalty=l1, score=nan, total=   0.0s
[CV] C=1, class_weight=None, penalty=l2 ..............................
[C

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)


[CV] .. C=1, class_weight=None, penalty=l2, score=0.081, total=   0.1s
[CV] C=1, class_weight=None, penalty=l2 ..............................
[CV] .. C=1, class_weight=None, penalty=l2, score=0.028, total=   0.1s
[CV] C=1, class_weight=None, penalty=l2 ..............................
[CV] .. C=1, class_weight=None, penalty=l2, score=0.054, total=   0.1s
CPU times: user 1.85 s, sys: 1.48 s, total: 3.33 s
Wall time: 1.76 s


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.7s finished


In [11]:
print(f" Best params are {logreg_gs.best_params_}")
      
print(f" F1 score on test is {round(f1_score(y_test, logreg_gs.best_estimator_.predict(x_test), average='macro')*100, 2)}")
print(f" F1 score on train is {round(f1_score(y_train, logreg_gs.best_estimator_.predict(x_train), average='macro')*100, 2)}")
print(f" Roc-auc score on test is {round(roc_auc_score(y_test, logreg_gs.best_estimator_.predict(x_test))*100, 2)}")
print(f" Roc-auc score on train is {round(roc_auc_score(y_train, logreg_gs.best_estimator_.predict(x_train))*100, 2)}")

 Best params are {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}
 F1 score on test is 42.0
 F1 score on train is 43.96
 Roc-auc score on test is 50.15
 Roc-auc score on train is 58.0


In [12]:
#  Grid search + Random Forest validation
rfc = RandomForestClassifier()
param_grid = {       
     'n_estimators' : [80, 160, 500, 1000],
     'criterion' : ['gini', 'entropy'],
     'max_depth' : [3, 5, 10],
     'min_samples_split' : [2, 3, 7]   
    }

rfc_gs = GridSearchCV(
    estimator=rfc,
    param_grid=param_grid,
    scoring=['f1'], 
    refit='f1',
    cv=5,
    verbose=4, 
    n_jobs=1,
)
rfc_gs.fit(x_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80, f1=0.000, total=   0.2s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80, f1=0.000, total=   0.2s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80 


...

[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80, f1=0.000, total=   0.2s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.7s remaining:    0.0s


[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80, f1=0.000, total=   0.3s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80 
[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=80, f1=0.000, total=   0.2s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160 
[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160, f1=0.000, total=   0.5s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160 
[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160, f1=0.000, total=   0.5s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160 
[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160, f1=0.000, total=   0.5s
[CV] criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160 
[CV]  criterion=gini, max_depth=3, min_samples_split=2, n_estimators=160, f1=0.000, total=   0.5s
[CV] criterion=gini, max_depth=3, min_samples_spli

[Parallel(n_jobs=1)]: Done 360 out of 360 | elapsed:  9.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [13]:
print(f" Best params are {rfc_gs.best_params_}")
print(f" F1 score on test is {round(f1_score(y_test, rfc_gs.best_estimator_.predict(x_test), average='macro')*100, 2)}")
print(f" F1 score on train is {round(f1_score(y_train, rfc_gs.best_estimator_.predict(x_train), average='macro')*100, 2)}")
print(f" Roc-auc score on test is {round(roc_auc_score(y_test, rfc_gs.best_estimator_.predict(x_test))*100, 2)}")
print(f" Roc-auc score on train is {round(roc_auc_score(y_train, rfc_gs.best_estimator_.predict(x_train))*100, 2)}")

 Best params are {'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 3, 'n_estimators': 80}
 F1 score on test is 48.82
 F1 score on train is 51.87
 Roc-auc score on test is 50.0
 Roc-auc score on train is 51.55


Xgboost + Grid Search


In [16]:
%time
xgbc = XGBClassifier()
param_grid = { 
    'binary':['logistic'],
     'n_estimators' : [80, 100, 150],
     'max_depth' : [3, 5, 7, 10],
     'learning_rate' : [0.1, 0.01, 0.001],
    'booster': ['gbtree', 'gblinear', 'dart'],
    'gamma': [0, 0.3, 0.5],
    'min_child_weight': [5, 6, 7, 8, 9, 10]
    }


xgbc_gs = GridSearchCV(
    estimator=xgbc,
    param_grid=param_grid,
    scoring="f1",
    cv=5, 
    verbose=4,
    n_jobs=1,
)
xgbc_gs.fit(x_train, y_train)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.58 µs
Fitting 5 folds for each of 1944 candidates, totalling 9720 fits
[CV] binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80, score=0.000, total=   0.6s
[CV] binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


[CV]  binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80, score=0.000, total=   0.4s
[CV] binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80 


...

[CV]  binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80, score=0.000, total=   0.3s
[CV] binary=logistic, booster=gbtree, gamma=0, learning_rate=0.1, max_depth=3, min_child_weight=5, n_estimators=80 


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.3s remaining:    0.0s


Выходные данные были обрезаны до нескольких последних строк (5000).
[CV] binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100 
[CV]  binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100, score=0.000, total=   0.3s
[CV] binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100 
[CV]  binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100, score=0.000, total=   0.3s
[CV] binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100 
[CV]  binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100, score=0.000, total=   0.3s
[CV] binary=logistic, booster=dart, gamma=0, learning_rate=0.001, max_depth=3, min_child_weight=6, n_estimators=100 
[CV]  binary=logistic, booste

[Parallel(n_jobs=1)]: Done 9720 out of 9720 | elapsed: 117.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_po...one,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=1,
             param_grid={'binary': ['logistic'],
                         'booster': ['gbtree', 'gblinear', 'dart'],
                         'gamma': [0, 0.3, 0.5],
                         'learning_rate': 

In [18]:
print(f" Best params are {xgbc_gs.best_params_}")
print(f" F1 score on test is {round(f1_score(y_test, xgbc_gs.best_estimator_.predict(x_test), average='macro')*100, 2)}")
print(f" F1 score on train is {round(f1_score(y_train, xgbc_gs.best_estimator_.predict(x_train), average='macro')*100, 2)}")
print(f" Roc-auc score on test is {round(roc_auc_score(y_test, xgbc_gs.best_estimator_.predict(x_test))*100, 2)}")
print(f" Roc-auc score on train is {round(roc_auc_score(y_train, xgbc_gs.best_estimator_.predict(x_train))*100, 2)}")

 Best params are {'binary': 'logistic', 'booster': 'gbtree', 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 10, 'min_child_weight': 5, 'n_estimators': 150}
 F1 score on test is 49.88
 F1 score on train is 57.67
 Roc-auc score on test is 50.49
 Roc-auc score on train is 54.8
